## Github Repo Metrics Using GraphQL API ##

text-processing.com provides a Sentiment Analysis API which has commercial licensing. 

It checks for neutrality first, ends if so, otherwise checks pos/neg

Vader is built on NLTK, open source, and worked previously for kono.

In [61]:
import requests
import json
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd
from string import Template

endpointSentAPI = 'http://text-processing.com/api/sentiment/'
endpointGitHub = 'https://api.github.com/graphql'
headersGitHub = {'Authorization': '<ADD YOUR TOKEN HERE>'}

def getSentimentAPI(phrase):
    r = requests.post(endpointSentAPI, {"text": phrase})
    return r.json()

def getRepoPRSentiment(ownerName, repoName):
    return


In [119]:
analyzer = SentimentIntensityAnalyzer()

repoQuery = Template(''' 
{ 
    repository(owner: \"$owner\", name: \"$repo\") { 
        forkCount
        watchers(first: 10) {
            totalCount
        }
        stargazers(first: 10) {
            totalCount
        }
        
        commitComments(first: 10) {
            edges {
                node {
                    bodyText
                    reactions(first: 10) {
                        edges {
                            node {
                                content
                            }
                        }
                    }
                }
            }
        }
        pullRequests(first: 10) { 
            totalCount
            edges { 
                node { 
                    createdAt
                    closedAt
                    closed
                    merged
                    mergedAt
                    comments(first: 10) {
                        edges {
                            node {
                                bodyText
                            }
                        }
                    } 
                } 
            } 
        } 
        issues(first: 10) {
            totalCount
            edges {
                node {
                    createdAt
                    closedAt
                    closed
                }
            }
        }
    }
}
''')

query = repoQuery.substitute(owner='eldritchjs',repo='equoid-data-handler')
r = requests.post(endpointGitHub, json.dumps({"query": query}), headers=headersGitHub)
data = r.json()['data']['repository']
repo = data['pullRequests']['edges']


In [120]:
for node in repo:
    edges = node['node']['comments']['edges']
    if len(edges) != 0:
        vaderNeg = 0
        vaderPos = 0
        vaderNeu = 0
        tpcNeg = 0
        tpcPos = 0
        tpcNeu = 0
        numMessages = 0
        for cnode in edges:
            phrase = cnode['node']['bodyText']
            numMessages+=1
            vaderScores = analyzer.polarity_scores(phrase)
            tpcScores = getSentimentAPI(phrase)

            tpcNeg += tpcScores['probability']['neg']
            tpcPos += tpcScores['probability']['pos']
            tpcNeu += tpcScores['probability']['neutral']
            vaderNeg += vaderScores['neg']
            vaderPos += vaderScores['pos']
            vaderNeu += vaderScores['neu']
            
        if(numMessages > 0):
            vaderNeg/=numMessages
            vaderPos/=numMessages
            vaderNeu/=numMessages
            vaderTotal = vaderNeg + vaderPos + vaderNeu
            vaderNeg = vaderNeg*100/vaderTotal
            vaderNeu = vaderNeu*100/vaderTotal
            vaderPos = vaderPos*100/vaderTotal
            
            tpcNeg/=numMessages
            tpcPos/=numMessages
            tpcNeu/=numMessages
            tpcTotal = tpcNeg + tpcPos + tpcNeu
            tpcNeg = tpcNeg*100/tpcTotal
            tpcNeu = tpcNeu*100/tpcTotal
            tpcPos = tpcPos*100/tpcTotal

            vaderNegStr = "%8.2f" % vaderNeg
            vaderPosStr = "%8.2f" % vaderPos
            vaderNeuStr = "%8.2f" % vaderNeu
            
            tpcNegStr = "%8.2f" % tpcNeg
            tpcPosStr = "%8.2f" % tpcPos
            tpcNeuStr = "%8.2f" % tpcNeu
            
            print('Vader - Negative: ' + vaderNegStr + ' Neutral: ' + vaderNeuStr + ' Positive: ' + vaderPosStr)
            print('T-p.c - Negative: ' + tpcNegStr + ' Neutral: ' + tpcNeuStr + ' Positive: ' + tpcPosStr)

Vader - Negative:     5.95 Neutral:    94.05 Positive:     0.00
T-p.c - Negative:    37.72 Neutral:    36.53 Positive:    25.75
Vader - Negative:     1.50 Neutral:    88.37 Positive:    10.13
T-p.c - Negative:    50.56 Neutral:    20.15 Positive:    29.29
